<img src='http://www-scf.usc.edu/~ghasemig/images/sharif.png' alt="SUT logo" width=300 height=300 align=left class="saturate" >

<br>
<font>
<div dir=ltr align=center>
<font color=0F5298 size=7>
    Artificial Intelligence <br>
<font color=2565AE size=5>
    Computer Engineering Department <br>
    Fall 2023<br>
<font color=3C99D size=5>
    Practical Assignment 1 - Search <br>
<font color=696880 size=4>
    Reza Soumi, Rouzbeh Pirayadi, Amir Mohammad Izadi

____

# Personal Data

In [1]:
# Set your student number
student_number = '400105069'
Name = 'محمدشایان'
Last_Name = 'شعبانی'

# Rules
- Make sure that all of your cells can be run perfectly. 

## Libraries

In [2]:
import pygame, time, csv
import numpy as np
from time import sleep
import heapq
from IPython.display import HTML

pygame 2.5.2 (SDL 2.28.3, Python 3.10.9)
Hello from the pygame community. https://www.pygame.org/contribute.html


# Uninformed & Informed Search (100 Points)

<font size=4>
Author: Reza Soumi
			<br/>
                <font color=red>
Please run all the cells.
     </font>
</font>
                <br/>
    </div>

## Problem Definition

In This Problem, We want to investigate Maze Problem with the focus on implementing IDS and A-Star algorithm to solve it. 

The concept of the **Digital Twin** was first introduced in 1960 during NASA's Apollo project. In this project, many simulation engines was created in the ground where sensory data from the spacecraft was sent to simulation devices. The model would then update itself with this data and send response or action commands back to the spacecraft using advanced telecommunication protocols. This concept has recently gained popularity.

If you are interested in learning more about digital twin in the Apollo NASA project, you can read this [link](https://blogs.sw.siemens.com/simcenter/apollo-13-the-first-digital-twin/) which explains how these systems helped astronauts return home safely. Additionally, there is a [TEDx](https://www.ted.com/talks/karen_willcox_how_digital_twins_could_help_us_predict_the_future?language=en) talk dedicated to this concept that you may find interesting.

Now, let's consider a scenario where we have launched a simple spacecraft into space without any IoT devices. Unfortunately, our robot has become stuck in a maze. The starting position is at the top left corner and the destination is at the bottom right corner, as shown in the picture below.

<!-- ![Sample Image](Pics/map.PNG) -->
<div style="text-align:center">
  <img src="Pics/map.PNG" alt="alt text" style="width:50%;"/>
</div>

To solve this problem, we need to implement an intelligent digital twin for the spacecraft. This digital twin will gather information from the spacecraft and determine which action to take - up, left, down, or right. However, we cannot explore the entire map at once and can only take one step forward at a time.

## Code

You can use the following data structures, and feel free to use any other data structures as needed, but these should suffice.

In [3]:
###########################################
###### no need to change any thing ########
###########################################

class Node:
    def __init__(self, pos, parent, cost=None):
        self.x = pos[0]
        self.y = pos[1]
        self.parent = parent
        self.cost = cost

    def position(self):
        return (self.x, self.y)

class Queue:
    def __init__(self):
        self.list = []

    def push(self,item):
        self.list.insert(0,item)

    def pop(self):
        return self.list.pop()

    def isEmpty(self):
        return len(self.list) == 0

class PriorityQueue:
    """ O(1) access to the lowest-priority item """
    def  __init__(self):
        self.heap = []
        self.count = 0

    def push(self, item, priority):
        entry = (priority, self.count, item)
        heapq.heappush(self.heap, entry)
        self.count += 1

    def pop(self):
        (_, _, item) = heapq.heappop(self.heap)
        return item

    def isEmpty(self):
        return len(self.heap) == 0

    def update(self, item, priority):
        # If item already in priority queue with higher priority, update its priority and rebuild the heap.
        for index, (p, c, i) in enumerate(self.heap):
            if i == item:
                if p <= priority:
                    break
                del self.heap[index]
                self.heap.append((priority, c, item))
                heapq.heapify(self.heap)
                break
        else:
            self.push(item, priority)

class Stack:
    def __init__(self):
        self.list = []

    def push(self,item):
        self.list.append(item)

    def pop(self):
        return self.list.pop()

    def isEmpty(self):
        return len(self.list) == 0

The code below takes a CSV file's path as the map address. In this CSV file, a value of 1 represents a passable block where the robot can navigate, while a value of 0 represents an impassable wall where the robot cannot go. There's no need to be concerned about Digital Twins at this point; we will implement them later.

In [4]:
###########################################
###### no need to change any thing ########
###########################################

def solve_maze(map_address, algorithm):
    grid = np.genfromtxt(map_address, delimiter=',', dtype=int)
    num_rows, num_columns = len(grid), len(grid[0])
    empty_block_count = np.count_nonzero(grid == 1)

    # Define start & goal positions
    start_pos = (0,0)
    goal_pos = (num_rows-1, num_columns-1)

    grid[0, 0] = 2
    grid[-1, -1] = 3

    grid_dim = (num_rows-1, num_columns-1)

    black, white, green, red, grey, blue, magenta = (0,0,0), (255, 255, 255), (50,205,50), (255,99,71), (211,211,211), (153,255,255), (255,0,255)
    idx_to_color = [black, white, green, red, blue, magenta]

    height = 15
    width = height
    margin = 1

    pygame.init()

    WINDOW_SIZE = [660, 660]
    screen = pygame.display.set_mode(WINDOW_SIZE)

    pygame.display.set_caption(f"{algorithm} Pathfinder. Solving: {map_address}")

    done = False
    run = False
    close = False

    clock = pygame.time.Clock()

    digital_twin = None

    if algorithm == "BFS":
        digital_twin = BFS_Digital_Twin(start_pos=start_pos, goal_pos=goal_pos, grid_dim=grid_dim)
    elif algorithm == "DFS":
        digital_twin = DFS_Digital_Twin(start_pos=start_pos, goal_pos=goal_pos, grid_dim=grid_dim)
    elif algorithm == "IDS":
        digital_twin = IDS_Digital_Twin(start_pos=start_pos, goal_pos=goal_pos, grid_dim=grid_dim)
    elif algorithm == "A_Star":
        digital_twin = A_Star_Digital_Twin(start_pos=start_pos, goal_pos=goal_pos, grid_dim=grid_dim)
    else:
        return None
    
    while not done:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                done = True

            elif event.type == pygame.KEYDOWN:
                run = True
                start_t0 = time.time()

        screen.fill(grey)

        for row in range(num_rows):
            for column in range(num_columns):
                color = idx_to_color[grid[row, column]]
                pygame.draw.rect(screen, color, [(margin + width) * column + margin, (margin + height) * row + margin, width, height])

        clock.tick(60)
        pygame.display.flip()

        if run == True:
            sleep(0.01)
            solution, done, grid = digital_twin.update(grid=grid)

        if done == True:
            print(f"Total empty block numbers: {empty_block_count}")
            print(f"Explored block numbers: {np.count_nonzero(grid == 4)}")
            for pos in solution:
                grid[pos[0], pos[1]] = 5

            screen.fill(grey)

            for row in range(num_rows):
                for column in range(num_columns):
                    color = idx_to_color[grid[row, column]]
                    pygame.draw.rect(screen, color, [(margin + width) * column + margin, (margin + height) * row + margin, width, height])

            clock.tick(60)
            pygame.display.flip()

    print(f"Your maze solved with {algorithm} algorithm.")
    print(f"--- finished {time.time()-start_t0:.3f} s---")
    while not close:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                close = True

            elif event.type == pygame.KEYDOWN:
                close = True
    pygame.quit()

Below is an implementation of our digital twin using the Depth-First Search (DFS) algorithm. Take a moment to explore how it functions.

In [7]:
###########################################
###### no need to change any thing ########
###########################################

class DFS_Digital_Twin:
    def __init__(self, start_pos, goal_pos, grid_dim):
        self.start_pos = start_pos
        self.goal_pos = goal_pos
        self.grid_dim = grid_dim
        self.stack = Stack()
        self.stack.push(Node(pos=start_pos, parent=None))

    def get_successors(self, x, y):
        return [(x+1, y), (x-1, y), (x, y+1), (x, y-1)]

    def is_valid_cell(self, pos):
        return 0 <= pos[0] <= self.grid_dim[0] and 0 <= pos[1] <= self.grid_dim[1]

    def backtrack_solution(self, curr_node):
        return self._backtrack(curr_node)

    def _backtrack(self, curr_node):
        return [] if curr_node.parent is None else self._backtrack(curr_node.parent) + [curr_node.position()]

    def update(self, grid):
        curr_state = self.stack.pop()
        x, y = curr_state.position()
        done = False
        solution_path = []

        for step in self.get_successors(x, y):
            if self.is_valid_cell(step) and grid[step[0], step[1]] in [1, 3]: # 1: empty cell has not explored yet, 3: goal cell
                self.stack.push(Node(pos=step, parent=curr_state))

                if step == self.goal_pos:
                    done = True
                    solution_path = self.backtrack_solution(curr_state)
                    break
            
            grid[x, y] = 4 # visited

        return solution_path, done, grid

You can run the code below to see how our exploration works (it is recommended to run it on your local machine instead of Colab).

In [8]:
solve_maze(map_address="mazes/maze_0.csv", algorithm="DFS")

Total empty block numbers: 881
Explored block numbers: 868
Your maze solved with DFS algorithm.
--- finished 14.294 s---


After running the code above and entering it on the screen, you should see the result, which resembles the GIF below.

In [9]:
gif_path = 'Pics/Animation.gif'
html_code = f'<div style="display: flex; justify-content: center;"> <img src="{gif_path}" alt="Your GIF" style="width: 40%;"/></div>'
HTML(html_code)

As you can see, all of the blocks have been explored using DFS algorithm. So, we need a better algorithm for this maze problem. You can think and devise a solution for this problem at this stage before proceeding.

Let's investigate other uninformed search methods. Your task now is to implement Breadth-First Search (BFS) algorithm to solve this maze problem. (Hint: You can simply just change the data structures used in the code provided above to convert DFS to BFS)

In [10]:
########################################################
######### implement BFS algorithm (10 Points) ##########
########################################################

class BFS_Digital_Twin:
    def __init__(self, start_pos, goal_pos, grid_dim):
        self.start_pos = start_pos
        self.goal_pos = goal_pos
        self.grid_dim = grid_dim
        self.queue = Queue()
        self.queue.push(Node(pos=start_pos, parent=None))
    
    def get_successors(self, x, y):
        return [(x+1, y), (x-1, y), (x, y+1), (x, y-1)]

    def is_valid_cell(self, pos):
        return 0 <= pos[0] <= self.grid_dim[0] and 0 <= pos[1] <= self.grid_dim[1]

    def backtrack_solution(self, curr_node):
        return self._backtrack(curr_node)

    def _backtrack(self, curr_node):
        return [] if curr_node.parent is None else self._backtrack(curr_node.parent) + [curr_node.position()]

    def update(self, grid):
        
        """
        Input: grid (2D array)
        Output:
            solution_path (List of tuples, empty if no solution found)
            done (Boolean, True if the goal is reached, False otherwise)
            grid (Updated 2D array)
        """
        
        done = False
        curr_state = self.queue.pop()
        x, y = curr_state.position()
        solution_path = []
        for step in self.get_successors(x, y):
            if self.is_valid_cell(step) and grid[step[0], step[1]] in [1, 3]:
                self.queue.push(Node(pos=step, parent=curr_state))
                if step == self.goal_pos:
                    done = True
                    solution_path = self.backtrack_solution(curr_state)
                    break
            grid[x, y] = 4
        
        return solution_path, done, grid

You can run the code below to see how BFS algorithm works for solving this maze problem.

In [11]:
solve_maze(map_address="mazes/maze_0.csv", algorithm="BFS")

Total empty block numbers: 881
Explored block numbers: 358
Your maze solved with BFS algorithm.
--- finished 5.930 s---


It seems that BFS algorithm provides a good solution for our maze problem. Now, let's try implementing the Iterative Deepening Search (IDS) algorithm in the code below. The depth must increase incrementally (one per request to the digital twin) according to the game policy.

In [5]:
########################################################
######### implement IDS algorithm (15 Points) ##########
########################################################

class IDS_Digital_Twin:
    def __init__(self, start_pos, goal_pos, grid_dim):
        self.start_pos = start_pos
        self.goal_pos = goal_pos
        self.grid_dim = grid_dim
        self.depth_limit = 1
        self.first = True
        self.stack = Stack()
        self.stack.push(Node(pos=start_pos, parent=None, cost=0))
    
    def get_successors(self, x, y):
        return [(x+1, y), (x-1, y), (x, y+1), (x, y-1)]

    def is_valid_cell(self, pos):
        return 0 <= pos[0] <= self.grid_dim[0] and 0 <= pos[1] <= self.grid_dim[1]

    def backtrack_solution(self, curr_node):
        return self._backtrack(curr_node)

    def _backtrack(self, curr_node):
        return [] if curr_node.parent is None else self._backtrack(curr_node.parent) + [curr_node.position()]
    
    def update(self, grid):
        if self.first:
            self.first = False
            self.grid = np.copy(grid)
        """
        Input: grid (2D array)
        Output:
            solution_path (List of tuples, empty if no solution found)
            done (Boolean, True if the goal is reached, False otherwise)
            grid (Updated 2D array)
        """
        if self.stack.isEmpty():
            self.depth_limit += 1
            grid = np.copy(self.grid)
            self.stack.push(Node(pos=self.start_pos, parent=None, cost=0))
                    
        curr_state = self.stack.pop()
        x, y = curr_state.position()
        done = False
        solution_path = []
        for step in self.get_successors(x, y):
            
            # if limit reached
            if curr_state.cost >= self.depth_limit:
                break
                
            if self.is_valid_cell(step) and grid[step[0], step[1]] in [1, 3]: # 1: empty cell has not explored yet, 3: goal cell
                self.stack.push(Node(pos=step, parent=curr_state, cost=curr_state.cost + 1))
                
                if step == self.goal_pos:
                    done = True
                    solution_path = self.backtrack_solution(curr_state)
                    break

            grid[x, y] = 4 # visited

        return solution_path, done, grid

In [6]:
solve_maze(map_address="mazes/maze_0.csv", algorithm="IDS")

Total empty block numbers: 881
Explored block numbers: 348
Your maze solved with IDS algorithm.
--- finished 807.349 s---


Question: 

So far, we have explored three uninformed search methods: DFS, BFS, and IDS. Briefly explain the advantages and disadvantages of each method in the context of the Maze Problem and compare their performance based on your observations (consider both time and memory). Suppose there is two destination points, address specific points based on this scenario.

Your Answer: (10 Points)
IDS and DFS order of usage of memory is linear, but BFS use an exponential order of memory
BFS is the fasted algorithm and returns the optimal path
DFS is the second fastest
Although IDS is not fast enough, but it will return the optimal path
Generally when we have memory limits, we should use IDS, and when there is no memeory limits, BFS is pretty good.

Question:

Provide a simple solution to improve the DFS algorithm's performance in terms of explored block numbers, Based on the game information.

Your Answer: (5 Points)
One simple improvement is to prefer right and below neighbors because based on the game information we know that the goal is located at the below right corner of the grid.

Now, let's turn our attention to a second maze map, 'maze_1.csv'. solve this maze using the BFS algorithm.

In [ ]:
solve_maze(map_address="mazes/maze_1.csv", algorithm="BFS")

As you can see, BFS may not always perform optimally, as it can end up exploring a majority of the blocks. Let's proceed by implementing the A-Star algorithm in the next cell.

In [5]:
########################################################
######### implement A* algorithm (50 Points) ###########
########################################################

def manhattan_distance(current_pos, goal_pos):
        x_diff = abs(current_pos[0] - goal_pos[0])
        y_diff = abs(current_pos[1] - goal_pos[1])
        return x_diff + y_diff

class A_Star_Digital_Twin:
    def __init__(self, start_pos, goal_pos, grid_dim):
        self.start_pos = start_pos
        self.goal_pos = goal_pos
        self.grid_dim = grid_dim
        self.priority_queue = PriorityQueue()
        self.priority_queue.push(Node(pos=start_pos, parent=None, cost=0), manhattan_distance(start_pos, goal_pos))
        
    def get_successors(self, x, y):
        return [(x+1, y), (x-1, y), (x, y+1), (x, y-1)]

    def is_valid_cell(self, pos):
        return 0 <= pos[0] <= self.grid_dim[0] and 0 <= pos[1] <= self.grid_dim[1]

    def backtrack_solution(self, curr_node):
        return self._backtrack(curr_node)

    def _backtrack(self, curr_node):
        return [] if curr_node.parent is None else self._backtrack(curr_node.parent) + [curr_node.position()]
    
    def update(self, grid):
        """
        Input: grid (2D array)
        Output:
            solution_path (List of tuples, empty if no solution found)
            done (Boolean, True if the goal is reached, False otherwise)
            grid (Updated 2D array)
        """
        
        done = False
        curr_state = self.priority_queue.pop()
        x, y = curr_state.position()
        solution_path = []
        for step in self.get_successors(x, y):
            if self.is_valid_cell(step) and grid[step[0], step[1]] in [1, 3]:
                self.priority_queue.push(Node(pos=step, parent=curr_state, cost=(1 + curr_state.cost)), manhattan_distance(step, self.goal_pos) + curr_state.cost + 1)
                if step == self.goal_pos:
                    done = True
                    solution_path = self.backtrack_solution(curr_state)
                    break
            grid[x, y] = 4
        
        return solution_path, done, grid
        
        

Now solve the 'maze_1.csv' with your A-star algorithm.

In [6]:
solve_maze(map_address="mazes/maze_1.csv", algorithm="A_Star")

Total empty block numbers: 881
Explored block numbers: 611
Your maze solved with A_Star algorithm.
--- finished 10.146 s---


Question:

Explain your heuristic and observation's result. Also, compare the A* method with Uniformed search in the context of the Maze problem.

Answer: 
The heuristic that I implemented is manhattan distance, obviously it is admissible because there is a lot of obstacles on the way to goal and we relaxed this. Using this heuristic we derived the exploration towards the below right corner of the grid where the goal state is located. So it is better than Uninformed search because we should move towards the goal, this reason makes A* faster and also gives us the optimal path (Heuristic is admissible).